# Data Science Capstone Project

This project is to practice all skills learn from the entire course and apply those knowledge to a near industrial project.

In [92]:
# !pip install geocoder
import geocoder

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

%load_ext sql
%matplotlib inline

# Enable IBM_db2 Database Access
%sql ibm_db_sa://trj52137:z89xgm4f30nmqh%5Eg@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB

print('Hello Capstone Project Course!')

Libraries imported.
The sql extension is already loaded. To reload it, use:
  %reload_ext sql
Hello Capstone Project Course!


In [93]:
# make sure to install these packages before running:
# pip install pandas
# !pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("datacatalog.cookcountyil.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(datacatalog.cookcountyil.gov,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("5pge-nu6u", limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
sale_2018 = results_df[results_df['sale_year']=='2018']
sale_2018 = sale_2018.reset_index()

In [94]:
sale_2018.head()

,index,addr,age,age_decade,age_decade_squared,age_squared,air,apts,attic_fnsh,attic_type,beds,bldg_sf,bsmt,bsmt_fin,centroid_x,centroid_y,class,cnst_qlty,condition_desirability_and_utility,condo_class_factor,condo_strata,doc_no,est_bldg,est_land,ext_wall,fbath,floodplain,frpl,gar1_area,gar1_att,gar1_cnst,gar1_size,gar2_area,gar2_att,gar2_cnst,gar2_size,garage_indicator,hbath,hd_sf,heat,improvement_size_squared,lot_size_squared,modeling_group,most_recent_sale,multi_code,multi_family_ind,multi_ind,n_units,nbhd,ncu,neigborhood_code_mapping_,o_hare_noise,oheat,ot_impr,per_ass,pin,porch,pure_market_filter,renovation,repair_cnd,road_proximity,roof_cnst,rooms,sale_date,sale_half_of_year,sale_half_year,sale_month_of_year,sale_price,sale_quarter,sale_quarter_of_year,sale_year,site,square_root_of_age,square_root_of_improvement_size,square_root_of_lot_size,total_bldg_sf,town_and_neighborhood,town_code,tp_dsgn,tp_plan,tractce,type_resd,use_1
0,1,4431 PRESCOTT AVE,57,5.7,32.49,3249,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,-87.8188917982924,41.8104051495936,299,NaN,NaN,299,1,01821519280,55890,4500,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19551,NaN,NaN,382241601,NCHARS,1,NaN,NaN,0,24,52,NaN,52,0,NaN,NaN,0.0439,18013090421010,3,1,NaN,NaN,1,NaN,NaN,2018-08-03T00:00:00.000,2,44,8,80000,87,3,2018,NaN,7.54983443527075,NaN,139.82489048806724,NaN,2152,21,NaN,NaN,819200,NaN,1
1,4,5313 N DELPHIA AVE,45,4.5,20.25,2025,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,-87.8408594548487,41.9758252818986,299,NaN,NaN,299,2,01807108202,74600,9750,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48470,NaN,NaN,2349340900,NCHARS,1,NaN,NaN,0,54,390,NaN,390,1,NaN,NaN,0.0183,12111190201042,3,1,NaN,NaN,0,NaN,NaN,2018-03-12T00:00:00.000,1,43,3,118000,85,1,2018,NaN,6.708203932499369,NaN,220.15903342811077,NaN,71390,71,NaN,NaN,760803,NaN,1
2,8,2221 N FREMONT ST,4,0.4,0.16000000000000003,16,1,0,0,3,5,4287,1,1,-87.6507348241407,41.9224566376035,208,2,NaN,NaN,NaN,01806815063,2529330,334800,2,4,0,2,2,2,2,4,0,0,0,7,1,3,3720,1,18378369,13838400,SF,1,2,NaN,0,NaN,12,0,12,0,5,0,NaN,14322110150000,3,1,NaN,2,0,6,9,2018-03-09T00:00:00.000,1,43,3,3705000,85,1,2018,2,2,65.47518613948341,60.991802727907626,NaN,7412,74,2,2,71100,3,1
3,24,4446 N PARKSIDE AVE,105,10.5,110.25,11025,0,0,3,1,3,1214,1,3,-87.7688628292223,41.9618626351786,203,2,NaN,NaN,NaN,01814944010,204800,56870,2,1,0,1,2,2,1,3,0,0,0,7,1,0,4375,2,1473796,19140625,SF,1,2,NaN,0,NaN,90,0,90,0,5,0,NaN,13172310190000,1,0,NaN,2,0,1,6,2018-05-29T00:00:00.000,1,43,5,1,86,2,2018,2,10.246950765959598,34.84250278036869,66.14378277661477,NaN,7190,71,0,0,150401,1,1
4,42,1926 PRAIRIE SQUARE,39,3.9,15.209999999999999,1521,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,-88.0392247604679,42.0639911332678,299,NaN,NaN,299,2,01823249087,79330,4460,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,252452,NaN,NaN,63732012304,NCHARS,1,NaN,NaN,0,226,100,NaN,100,0,NaN,NaN,0.005442,07122000091176,3,1,NaN,NaN,0,NaN,NaN,2018-08-20T00:00:00.000,2,44,8,132000,87,3,2018,NaN,6.244997998398398,NaN,502.44601700083166,NaN,35100,35,NaN,NaN,804603,NaN,1


In [126]:
hardship = %sql SELECT hardship_index, community_area_name, community_area_number, percent_aged_25__without_high_school_diploma, per_capita_income FROM CENSUS_DATA ORDER BY hardship_index NULLS LAST;
columns = ['hardship_index', 'community_area_name', 'community_area_number', 'percent_aged_25__without_high_school_diploma', 'per_capita_income']
hardship = pd.DataFrame(hardship, columns = columns)
chicago_avg = hardship[hardship['hardship_index'].isnull()]
# print(hardship.shape)
hardship.dropna(axis = 0, subset = ['community_area_number'],inplace = True)
# print(hardship.shape)

 * ibm_db_sa://trj52137:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


In [122]:
hardship.head()

In [101]:
census = %sql SELECT * FROM CENSUS_DATA;
columns = ['community_area_number','community_area_name','percent_of_housing_crowded','percent_households_below_poverty', 'percent_aged_16__unemployed','percent_aged_25__without_high_school_diploma','percent_aged_under_18_or_over_64','per_capita_income','hardship_index']
census = pd.DataFrame(census, columns = columns)
print(census.shape)
census.dropna(axis = 0, subset = ['community_area_number'],inplace = True)
print(census.shape)
census['community_area_number'] = census['community_area_number'].astype(int)

 * ibm_db_sa://trj52137:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.
(78, 9)
(77, 9)


In [102]:
census.head()

,community_area_number,community_area_name,percent_of_housing_crowded,percent_households_below_poverty,percent_aged_16__unemployed,percent_aged_25__without_high_school_diploma,percent_aged_under_18_or_over_64,per_capita_income,hardship_index
0,1,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
1,2,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46.0
2,3,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20.0
3,4,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17.0
4,5,North Center,0.3,7.5,5.2,4.5,26.2,57123,6.0


In [105]:
census = census.sort_values(by='hardship_index').reset_index(drop = True)

In [107]:
census_sorted = census[['hardship_index','community_area_name','community_area_number','per_capita_income','percent_aged_25__without_high_school_diploma']]